In [2]:
import numpy as np
import control as ctrl

from lib.OrderUncertainties import *
from src.Benchmarks import Bench, sys_variables


In [4]:
def get_sensitivity(A, B):
    mat = OrdUnc(A)
    if mat.determineCase() == 1:
        unc = mat.distinctPos(B)
    else:
        unc = mat.multSig(B)
    return unc

def all_sensitivity(bench, K, openloop=False):
    # A_bar_i = A + BKE_i where E_i = I(n) with the (i, i) entry swapped to lambda
    A = bench.A + bench.B * K
    B = bench.B * K
    sensitivity = []
    for i in range(bench.nx):
        Bi = np.zeros((bench.nx, bench.nx))
        Bi[:, i] = B[:, i]
        sensitivity.append(get_sensitivity(A, Bi))

    return sensitivity


In [10]:
res = {}
for (name, bench) in sys_variables.items():
    K = ctrl.lqr(bench.sysd, np.eye(bench.nx), np.eye(bench.nu))[0]
    res[name] = list(map(lambda x: float(f"{x:.2g}"), all_sensitivity(bench, K)))
res

{'RC': [-0.059, 0.24],
 'F1': [8.5, -2.4],
 'DC': [0.00014, -0.00092],
 'CS': [54.0, 12.0, 16.0, 0.4],
 'EW': [9700.0, -3.4e-05],
 'C1': [-0.00099],
 'CC': [0.066, 0.14, -0.19]}